In [3]:
# this code belongs to this repository:
# https://github.com/ludobouan/pure-numpy-feedfowardNN


%matplotlib inline

import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt

np.seterr(over='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [17]:
class NeuralNetwork():
    def __init__(self):
        np.random.seed(1)  # Seed the random number generator
        self.weights = {}  # Create dict to hold weights
        self.num_layers = 1  # Set initial number of layer to one (input layer)
        self.adjustments = {}  # Create dict to hold the derivates of W of each layer

    def add_layer(self, shape):
        # Create weights with shape specified + biases
        # the last array has the bias
        ################################################################################################## this part are the bias
        self.weights[self.num_layers] = np.vstack((2 * np.random.random(shape) - 1, 2 * np.random.random((1, shape[1])) - 1))
        # Initialize the adjustements for these weights to zero
        self.adjustments[self.num_layers] = np.zeros(shape)
        self.num_layers += 1

    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, data):
        # Pass data through pretrained network
        for layer in range(1, self.num_layers+1):
            data = np.dot(data, self.weights[layer-1][:, :-1]) + self.weights[layer-1][:, -1] # + self.biases[layer]
            data = self.__sigmoid(data)
        return data

    def __forward_propagate(self, data):
        # Progapagate through network and hold values for use in back-propagation
        activation_values = {} # acumulates the output of each layer, incluing the output of x(input) that would be the output of layer 1
        activation_values[1] = data
        for layer in range(2, self.num_layers+1):
            # z = (X . W) + B
            print("------------------------")
            print("self.num_layers:",self.num_layers)
            print(self.weights)
            print(self.weights[layer-1][:-1, :])
            print("------------------------")
            data = np.dot(data.T, self.weights[layer-1][:-1, :]) + self.weights[layer-1][-1, :].T # + self.biases[layer]
            # A = g(Z)
            data = self.__sigmoid(data).T
            activation_values[layer] = data
        return activation_values

    def simple_error(self, outputs, targets):
        return targets - outputs

    def sum_squared_error(self, outputs, targets):
        return 0.5 * np.mean(np.sum(np.power(outputs - targets, 2), axis=1))

    def __back_propagate(self, output, target):
        """
        print("Debuging back propagation...")
        print("last layer------------------")
        print("target",target)
        print("output", output)
        print("output[self.num_layers]", output[self.num_layers], self.num_layers)
        #"""
        deltas = {}
        
        # Delta of output Layer-------------------------------------------------------------------------
        # dz3 = a3 - y
        deltas[self.num_layers] = output[self.num_layers] - target # the output less the target | dz
        #print("deltas", deltas)
        #-----------------------------------------------------------------------------------------------
        
        # Delta of hidden Layers------------------------------------------------------------------------
        for layer in reversed(range(2, self.num_layers)):  # All layers except input/output
            #print("hindden layers------------------")
            a_val = output[layer] #output of layer 2 (in this case)
            #print("a_val", a_val, 33)
            #print("self.weights", self.weights)
            weights = self.weights[layer][:-1, :] # weights of layer 2 (in this case)
            #print("weights", weights)
            prev_deltas = deltas[layer+1]#delta ouput of layer 3 (in this case)
            #print("prev_deltas", prev_deltas)
            # dz2 =  (W3.T . dz3) * g'(a2)
            deltas[layer] = np.multiply(np.dot(weights, prev_deltas), self.__sigmoid_derivative(a_val)) # delta of w of layer 2
            #print("deltas", deltas) # delta contains derivates of all outputs
        #----------------------------------------------------------------------------------------------

        # Caclculate total adjustements based on deltas-------------------------------------------------
        """
        print("Caclculate total adjustements based on deltas------------------")
        print("self.adjustments", self.adjustments)
        """
        for layer in range(1, self.num_layers):
            """
            print("layer", layer,"##")
            print("output[layer].T", output[layer].T)
            print("deltas[layer+1]", deltas[layer+1])
            """
            #dw2 = dz3 . a2.T
            #dw1 = dz1 . x.T
            #print("np.dot(deltas[layer+1], output[layer].T).T", np.dot(deltas[layer+1], output[layer].T).T, 666666)
            self.adjustments[layer] += np.dot(deltas[layer+1], output[layer].T).T
            #print("self.adjustments[layer]", self.adjustments[layer])

    def __gradient_descente(self, batch_size, learning_rate):
        # Calculate partial derivative and take a step in that direction
        """
        print("Debuging gradient descent...........................")
        print("batch_size:",batch_size)
        print("learning_rate:",learning_rate)
        print("self.adjustments", self.adjustments)
        #"""
        for layer in range(1, self.num_layers):
            #print("layer", layer, "###########")
            #print("self.adjustments[layer]", self.adjustments[layer])
            partial_d = (1/batch_size) * self.adjustments[layer]
            #print("partial_d", partial_d)
            #print("partial_d[-1, :]",partial_d[-1, :])
            self.weights[layer][:-1, :] += learning_rate * -partial_d
            self.weights[layer][-1, :] += learning_rate*1e-3 * -partial_d[-1, :]


    def train(self, inputs, targets, num_epochs, learning_rate=1, stop_accuracy=1e-5):
        error = []
        for iteration in range(num_epochs):
            #print("------------------------")
            for i in range(len(inputs)):
                x = inputs[i]
                y = targets[i]
                print("###########")
                print(i)
                print("current data:",x)
                print("###########")
                # Pass the training set through our neural network
                output = self.__forward_propagate(x)

                # Calculate the error
                loss = self.sum_squared_error(output[self.num_layers], y)
                error.append(loss)

                # Calculate Adjustements
                self.__back_propagate(output, y)

            self.__gradient_descente(i, learning_rate)

            # Check if accuarcy criterion is satisfied
            """
            print("error[-(i+1):]", error[-(i+1):])
            print("np.mean(error[-(i+1):])", np.mean(error[-(i+1):]))
            print("if", np.mean(error[-(i+1):]) < stop_accuracy and iteration > 0)
            print(error)
            """
            if np.mean(error[-(i+1):]) < stop_accuracy and iteration > 0:
                break

        return(np.asarray(error), iteration+1)

In [18]:
# Create instance of a neural network
nn = NeuralNetwork()

# Add Layers (Input layer is created by default)

nn.add_layer((2, 9))
nn.add_layer((9, 1))

# XOR function
training_data = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]]).reshape(4, 2, 1)
training_labels = np.asarray([[0], [1], [1], [0]])
error, iteration = nn.train(training_data, training_labels, 5000)
print('Error = ', np.mean(error[-4:]))
print('Epoches needed to train = ', iteration)
# nn.predict(testing_data)

###########
0
current data: [[0]
 [0]]
###########
------------------------
self.num_layers: 3
{1: array([[-0.16595599,  0.44064899, -0.99977125, -0.39533485, -0.70648822,
        -0.81532281, -0.62747958, -0.30887855, -0.20646505],
       [ 0.07763347, -0.16161097,  0.370439  , -0.5910955 ,  0.75623487,
        -0.94522481,  0.34093502, -0.1653904 ,  0.11737966],
       [-0.71922612, -0.60379702,  0.60148914,  0.93652315, -0.37315164,
         0.38464523,  0.7527783 ,  0.78921333, -0.82991158]]), 2: array([[-0.92189043],
       [-0.66033916],
       [ 0.75628501],
       [-0.80330633],
       [-0.15778475],
       [ 0.91577906],
       [ 0.06633057],
       [ 0.38375423],
       [-0.36896874],
       [ 0.37300186]])}
[[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822 -0.81532281
  -0.62747958 -0.30887855 -0.20646505]
 [ 0.07763347 -0.16161097  0.370439   -0.5910955   0.75623487 -0.94522481
   0.34093502 -0.1653904   0.11737966]]
------------------------
--------------------

###########
2
current data: [[1]
 [0]]
###########
------------------------
self.num_layers: 3
{1: array([[-0.51773153,  0.06374881, -0.86726605, -0.51500512, -0.63523613,
         0.33181426, -0.69508042, -0.29962881, -0.29302472],
       [-0.40135141,  0.06339535, -0.18899622, -0.58307402,  0.70575717,
        -0.17015119,  0.3631501 , -0.3210639 , -0.08986689],
       [-0.71970511, -0.60357202,  0.6009297 ,  0.93653117, -0.37320212,
         0.3854203 ,  0.75280052,  0.78905765, -0.83011882]]), 2: array([[-0.92503239],
       [-0.38649279],
       [ 0.27777787],
       [-0.61296454],
       [ 0.11285563],
       [ 0.13246448],
       [-0.38461965],
       [ 0.41302845],
       [-0.35292131],
       [ 0.3730179 ]])}
[[-0.51773153  0.06374881 -0.86726605 -0.51500512 -0.63523613  0.33181426
  -0.69508042 -0.29962881 -0.29302472]
 [-0.40135141  0.06339535 -0.18899622 -0.58307402  0.70575717 -0.17015119
   0.3631501  -0.3210639  -0.08986689]]
------------------------
--------------------

###########
1
current data: [[0]
 [1]]
###########
------------------------
self.num_layers: 3
{1: array([[-1.85600669, -0.24712274, -0.78477744, -0.44344311, -0.90421112,
         1.335862  , -0.88683599, -0.59278836, -0.43814667],
       [-2.08533973,  0.37532892, -0.89026628, -0.45385913,  0.59879554,
         0.40979506,  1.0178669 , -0.74490874, -0.52186109],
       [-0.7213891 , -0.60326008,  0.60022843,  0.93666039, -0.37330908,
         0.38600025,  0.75345524,  0.78863381, -0.83055082]]), 2: array([[-1.66063288],
       [-0.03584959],
       [ 0.12383872],
       [ 0.30892859],
       [ 0.96629606],
       [-0.49348736],
       [-1.11725008],
       [ 0.85890364],
       [-0.2964686 ],
       [ 0.37307436]])}
[[-1.85600669 -0.24712274 -0.78477744 -0.44344311 -0.90421112  1.335862
  -0.88683599 -0.59278836 -0.43814667]
 [-2.08533973  0.37532892 -0.89026628 -0.45385913  0.59879554  0.40979506
   1.0178669  -0.74490874 -0.52186109]]
------------------------
----------------------

------------------------
self.num_layers: 3
{1: array([[-5.15503425, -0.60903271, -0.67785297, -0.58962346, -2.84005947,
         2.05091122, -2.52375159, -0.7299914 , -0.99280246],
       [-5.44685142,  0.6981366 , -1.56524073, -0.35173428,  1.59369419,
        -0.38212223,  4.16853017, -0.65084711, -1.3648123 ],
       [-0.72475061, -0.60293727,  0.59955346,  0.93676251, -0.37231418,
         0.38520833,  0.7566059 ,  0.78872787, -0.83139377]]), 2: array([[-7.23403044],
       [ 0.70340006],
       [-0.34461876],
       [ 1.76853362],
       [ 3.16983581],
       [-0.1828636 ],
       [-3.52419639],
       [ 1.59090254],
       [-0.81404216],
       [ 0.37255678]])}
[[-7.23403044]
 [ 0.70340006]
 [-0.34461876]
 [ 1.76853362]
 [ 3.16983581]
 [-0.1828636 ]
 [-3.52419639]
 [ 1.59090254]
 [-0.81404216]]
------------------------
###########
0
current data: [[0]
 [0]]
###########
------------------------
self.num_layers: 3
{1: array([[-5.44342333, -0.66241021, -0.67831703, -0.65007227, -3.

self.num_layers: 3
{1: array([[-9.23183005, -1.36401415, -1.07225122, -0.78243572, -8.1221897 ,
         2.819271  , -6.20003266, -0.40620839, -2.31700761],
       [-9.5172505 ,  1.27743556, -2.60595166, -0.24558854,  4.62614025,
        -2.09572046, 10.49483877,  0.23648528, -2.82388704],
       [-0.72882101, -0.60235798,  0.59851275,  0.93686866, -0.36928174,
         0.38349474,  0.76293221,  0.7896152 , -0.83285284]]), 2: array([[-17.44081596],
       [  2.31801195],
       [ -1.05858194],
       [  4.14151767],
       [  8.75690609],
       [  0.36127272],
       [ -9.82843958],
       [  3.05548574],
       [ -2.37132631],
       [  0.3709995 ]])}
[[-9.23183005 -1.36401415 -1.07225122 -0.78243572 -8.1221897   2.819271
  -6.20003266 -0.40620839 -2.31700761]
 [-9.5172505   1.27743556 -2.60595166 -0.24558854  4.62614025 -2.09572046
  10.49483877  0.23648528 -2.82388704]]
------------------------
------------------------
self.num_layers: 3
{1: array([[-9.23183005, -1.36401415, -1.072

In [9]:
r = np.array([[1,  0.44064899, -0.99977125, -0.39533485, -0.70648822,
        -0.81532281, -0.62747958, -0.30887855, -0.20646505],
       [ 2, -0.16161097,  0.370439  , -0.5910955 ,  0.75623487,
        -0.94522481,  0.34093502, -0.1653904 ,  0.11737966],
       [3, -0.60379702,  0.60148914,  0.93652315, -0.37315164,
         0.38464523,  0.7527783 ,  0.78921333, -0.82991158]])
r[:-1, :]

array([[ 1.        ,  0.44064899, -0.99977125, -0.39533485, -0.70648822,
        -0.81532281, -0.62747958, -0.30887855, -0.20646505],
       [ 2.        , -0.16161097,  0.370439  , -0.5910955 ,  0.75623487,
        -0.94522481,  0.34093502, -0.1653904 ,  0.11737966]])

In [10]:
r = [1,2,3,4,5,6,7,8,9]

r[-4:]

[6, 7, 8, 9]

In [ ]:
r = np.array([[1,  0.44064899, -0.99977125, -0.39533485, -0.70648822,
        -0.81532281, -0.62747958, -0.30887855, -0.20646505]]).T
r[:-1, :]

In [ ]:

1e-5

In [2]:
print(7.299670911230132e-06<1)

True


In [ ]:
-5.04310222e-03 == -0.0050431